In [2]:
import os

In [5]:
%pwd

'/home/sohamp/Workspace/somsom/text_summarizer'

In [4]:
os.chdir("../")

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from text_summarizer.constants import *
from text_summarizer.utils.common import read_yaml,create_directories


In [8]:
class ConfigurationManager:
    def __init__(
            self,
            config_file_path: Path = CONFIG_FILE_PATH,
            params_file_path: Path = PARAMS_FILE_PATH
            ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)->DataIngestionConfig:
        config=self.config.data_ingestion

        create_directories([config.root_dir])

        data_Ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_Ingestion_config

In [9]:
import os
import urllib.request as request
import zipfile
from text_summarizer.logging import logger
from text_summarizer.utils.common import get_size



In [10]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config=config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            fileName, headers=request.urlretrieve(
                url=self.config.source_url,
                filename=self.config.local_data_file
            )
            logger.info(f"Downloaded file: {fileName} of size: {get_size(fileName)}")
        else:
            logger.info(f"File already present at {self.config.local_data_file}")

    def unzip_file(self):
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,"r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_file()
except Exception as e:
    raise e

2023-09-20 02:06:04,115: INFO: common: Loaded yaml file from config/config.yaml
2023-09-20 02:06:04,120: INFO: common: Loaded yaml file from params.yaml
2023-09-20 02:06:04,122: INFO: common: Created directory artifacts
2023-09-20 02:06:04,127: INFO: common: Created directory artifacts/data_ingestion
2023-09-20 02:06:04,129: INFO: 966062860: File already present at artifacts/data_ingestion/summarizer-data.zip
